In [1]:
#impor packages
import pandas as pd
import numpy as np
import os
import pickle
import json
import re

In [2]:
# location of raw data files
input_path = "../datasets/raw/"

In [3]:
#read LABEVENTS.csv file
df = pd.read_csv(filepath_or_buffer=input_path+'MICROBIOLOGYEVENTS.csv',dtype = {'SUBJECT_ID':"object", 
                                                                                 'HADM_ID':"object", 
                                                                                 'CHARTDATE':"object", 
                                                                                 'CHARTTIME':"object", 
                                                                                 'SPEC_ITEMID':"object",
                                                                                 'SPEC_TYPE_DESC':"object", 
                                                                                 'ORG_ITEMID':"object", 
                                                                                 'ORG_NAME':"object", 
                                                                                 'ISOLATE_NUM':"object", 
                                                                                 'AB_ITEMID':"object",
                                                                                 'AB_NAME':"object", 
                                                                                 'DILUTION_TEXT':"object", 
                                                                                 'DILUTION_COMPARISON':"object", 
                                                                                 'DILUTION_VALUE':"object",
                                                                                 'INTERPRETATION':"object"})

In [4]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,AB_ITEMID,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION
0,744,96,170324,2156-04-13 00:00:00,2156-04-13 14:18:00,70021,BRONCHOALVEOLAR LAVAGE,80026,PSEUDOMONAS AERUGINOSA,1,NaN,NaN,NaN,NaN,NaN,NaN
1,745,96,170324,2156-04-20 00:00:00,2156-04-20 13:10:00,70062,SPUTUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,746,96,170324,2156-04-20 00:00:00,2156-04-20 16:00:00,70012,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,747,96,170324,2156-04-20 00:00:00,NaN,70012,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,748,96,170324,2156-04-20 00:00:00,NaN,70079,URINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Dropping Row_ID 
df = df.drop(labels='ROW_ID',axis = 1)

In [6]:
# Print datatype and non-null data points for each column
df.info(verbose=True,null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631726 entries, 0 to 631725
Data columns (total 15 columns):
SUBJECT_ID             631726 non-null object
HADM_ID                631726 non-null object
CHARTDATE              631726 non-null object
CHARTTIME              589954 non-null object
SPEC_ITEMID            631647 non-null object
SPEC_TYPE_DESC         631726 non-null object
ORG_ITEMID             326881 non-null object
ORG_NAME               328016 non-null object
ISOLATE_NUM            328016 non-null object
AB_ITEMID              275834 non-null object
AB_NAME                275834 non-null object
DILUTION_TEXT          267350 non-null object
DILUTION_COMPARISON    267178 non-null object
DILUTION_VALUE         267178 non-null object
INTERPRETATION         275834 non-null object
dtypes: object(15)
memory usage: 72.3+ MB


In [7]:
# Replacing missing values in ORG_NAME with "Negative Culture"
df['ORG_NAME'] = df['ORG_NAME'].fillna('Negative Culture')

In [8]:
df.head()

,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,AB_ITEMID,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION
0,96,170324,2156-04-13 00:00:00,2156-04-13 14:18:00,70021,BRONCHOALVEOLAR LAVAGE,80026,PSEUDOMONAS AERUGINOSA,1,NaN,NaN,NaN,NaN,NaN,NaN
1,96,170324,2156-04-20 00:00:00,2156-04-20 13:10:00,70062,SPUTUM,NaN,Negative Culture,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,96,170324,2156-04-20 00:00:00,2156-04-20 16:00:00,70012,BLOOD CULTURE,NaN,Negative Culture,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,96,170324,2156-04-20 00:00:00,NaN,70012,BLOOD CULTURE,NaN,Negative Culture,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,96,170324,2156-04-20 00:00:00,NaN,70079,URINE,NaN,Negative Culture,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Dropping ITEM id columns as the text description of these columns is available
df = df.drop(columns=['CHARTDATE','SPEC_ITEMID','ORG_ITEMID','AB_ITEMID'])

In [10]:
df.head()

,SUBJECT_ID,HADM_ID,CHARTTIME,SPEC_TYPE_DESC,ORG_NAME,ISOLATE_NUM,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION
0,96,170324,2156-04-13 14:18:00,BRONCHOALVEOLAR LAVAGE,PSEUDOMONAS AERUGINOSA,1,NaN,NaN,NaN,NaN,NaN
1,96,170324,2156-04-20 13:10:00,SPUTUM,Negative Culture,NaN,NaN,NaN,NaN,NaN,NaN
2,96,170324,2156-04-20 16:00:00,BLOOD CULTURE,Negative Culture,NaN,NaN,NaN,NaN,NaN,NaN
3,96,170324,NaN,BLOOD CULTURE,Negative Culture,NaN,NaN,NaN,NaN,NaN,NaN
4,96,170324,NaN,URINE,Negative Culture,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Reomving Special Charecters from SPEC_TYPE_DESC and ORG_NAME
df['SPEC_TYPE_DESC'] = df['SPEC_TYPE_DESC'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x)) 
df['ORG_NAME'] = df['ORG_NAME'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x)) 

In [12]:
# combine Anitbiotic Name and its dilution value
df['AB_NAME_DILUTION'] = df['AB_NAME']+'-'+df['DILUTION_TEXT']

In [13]:
# Droping columns which are not required as their data exists in columns being retained
df = df.drop(labels = ['ISOLATE_NUM','AB_NAME','DILUTION_TEXT','DILUTION_COMPARISON','DILUTION_VALUE'], axis = 'columns')

In [14]:
# Fill missing values with 'NaN'
df = df.fillna('NaN')

In [15]:
df.head(100)

,SUBJECT_ID,HADM_ID,CHARTTIME,SPEC_TYPE_DESC,ORG_NAME,INTERPRETATION,AB_NAME_DILUTION
0,96,170324,2156-04-13 14:18:00,BRONCHOALVEOLAR LAVAGE,PSEUDOMONAS AERUGINOSA,NaN,NaN
1,96,170324,2156-04-20 13:10:00,SPUTUM,Negative Culture,NaN,NaN
2,96,170324,2156-04-20 16:00:00,BLOOD CULTURE,Negative Culture,NaN,NaN
3,96,170324,NaN,BLOOD CULTURE,Negative Culture,NaN,NaN
4,96,170324,NaN,URINE,Negative Culture,NaN,NaN
5,96,170324,2156-04-21 14:00:00,SPUTUM,Negative Culture,NaN,NaN
6,101,175533,NaN,URINE,Negative Culture,NaN,NaN
7,101,175533,2196-09-27 00:00:00,BLOOD CULTURE,Negative Culture,NaN,NaN
8,101,175533,2196-09-27 00:00:00,BLOOD CULTURE,Negative Culture,NaN,NaN
9,101,175533,2196-09-27 00:00:00,SPUTUM,Negative Culture,NaN,NaN


In [16]:
# Combine all columns into a single column
df['microbioevents'] = df['SPEC_TYPE_DESC']+' '+df['ORG_NAME']+' '+df['AB_NAME_DILUTION']+' '+df['INTERPRETATION']

In [17]:
# Drop columns with redundant info
df = df.drop(labels=['SPEC_TYPE_DESC','ORG_NAME','AB_NAME_DILUTION','INTERPRETATION'], axis = 'columns')

In [18]:
# Drop rows with missing Charttime 
df = df[df['CHARTTIME']!='NaN'].reset_index(drop = True)

In [19]:
df.head(100)

,SUBJECT_ID,HADM_ID,CHARTTIME,microbioevents
0,96,170324,2156-04-13 14:18:00,BRONCHOALVEOLAR LAVAGE PSEUDOMONAS AERUGINOSA ...
1,96,170324,2156-04-20 13:10:00,SPUTUM Negative Culture NaN NaN
2,96,170324,2156-04-20 16:00:00,BLOOD CULTURE Negative Culture NaN NaN
3,96,170324,2156-04-21 14:00:00,SPUTUM Negative Culture NaN NaN
4,101,175533,2196-09-27 00:00:00,BLOOD CULTURE Negative Culture NaN NaN
5,101,175533,2196-09-27 00:00:00,BLOOD CULTURE Negative Culture NaN NaN
6,101,175533,2196-09-27 00:00:00,SPUTUM Negative Culture NaN NaN
7,101,175533,2196-09-27 00:01:00,URINE Negative Culture NaN NaN
8,101,175533,2196-09-28 20:50:00,SPUTUM Negative Culture NaN NaN
9,101,175533,2196-09-28 20:50:00,BLOOD CULTURE Negative Culture NaN NaN


In [20]:
# Split the each event by whitespace
df['microbioevents'] = df['microbioevents'].map(lambda x: x.split())

In [21]:
# remove 'NaN' from all events
df['microbioevents'] = df['microbioevents'].map(lambda x: list(filter(lambda a: a!='NaN',x)))

In [22]:
df.head(100)

,SUBJECT_ID,HADM_ID,CHARTTIME,microbioevents
0,96,170324,2156-04-13 14:18:00,"[BRONCHOALVEOLAR, LAVAGE, PSEUDOMONAS, AERUGIN..."
1,96,170324,2156-04-20 13:10:00,"[SPUTUM, Negative, Culture]"
2,96,170324,2156-04-20 16:00:00,"[BLOOD, CULTURE, Negative, Culture]"
3,96,170324,2156-04-21 14:00:00,"[SPUTUM, Negative, Culture]"
4,101,175533,2196-09-27 00:00:00,"[BLOOD, CULTURE, Negative, Culture]"
5,101,175533,2196-09-27 00:00:00,"[BLOOD, CULTURE, Negative, Culture]"
6,101,175533,2196-09-27 00:00:00,"[SPUTUM, Negative, Culture]"
7,101,175533,2196-09-27 00:01:00,"[URINE, Negative, Culture]"
8,101,175533,2196-09-28 20:50:00,"[SPUTUM, Negative, Culture]"
9,101,175533,2196-09-28 20:50:00,"[BLOOD, CULTURE, Negative, Culture]"


In [23]:
# create output path
mypath_output = "../datasets/microbioevents/tokenized/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [24]:
# export the dataframe to JSON format
df.to_json(mypath_output+"microbioevents.json",orient = 'records') 